In [ ]:
!pip install -U sentence-transformers
!pip install scikit-learn-extra
!pip install -q pyyaml h5py
!pip install delayed

     |████████████████████████████████| 85 kB 3.6 MB/s 
     |████████████████████████████████| 2.6 MB 26.2 MB/s 
     |████████████████████████████████| 1.2 MB 43.3 MB/s 
     |████████████████████████████████| 43 kB 909 kB/s 
     |████████████████████████████████| 636 kB 43.6 MB/s 
     |████████████████████████████████| 895 kB 39.9 MB/s 
     |████████████████████████████████| 3.3 MB 44.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=0540c06605cb66ba7b22f0eb1b9224fc3a1a4429b6b47e03e3ab901486e68680
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.7 MB 12.5 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
  At

In [ ]:
# print(tf.__version__)
# from google.colab import drive 
# drive.mount('/content/drive')


1.15.2
Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from string import punctuation
import re
import pickle
from sklearn_extra.cluster import KMedoids
import pandas as pd
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
import nltk, re, time
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from collections import namedtuple
import os


TensorFlow 1.x selected.
1.15.2


Using TensorFlow backend.


# Data preprocessing

In [ ]:
def clean_text(text, remove_stopwords=True):
	'''Clean the text, with the option to remove stopwords'''

	# Convert words to lower case and split them
	text = text.lower().split()

	# Optionally, remove stop words
	if remove_stopwords:
		# stops = set(stopwords.words("english"))
		stops = set(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y'])
		text = [w for w in text if not w in stops]

	text = " ".join(text)

	# Clean the text
	text = re.sub(r"<br />", " ", text)
	text = re.sub(r"[^a-z.]", " ", text)
	text = re.sub(r"   ", " ", text)  # Remove any extra spaces
	text = re.sub(r"  ", " ", text)
	# Remove punctuation from text
	text = ''.join([c for c in text if (c not in punctuation) or (c == '.')])

	return (text)

In [ ]:
import nltk
nltk.download('stopwords')

ipf = './'
opf = "./"

if not os.path.exists(opf):
    os.makedirs(opf)
# Load the data

train_data = pd.read_csv(ipf + "train.csv",delimiter='\t')
max_sent_len = 0
max_sent_count = 0
sent_lens = []
sent_counts = []
train_data1 = []
y_train_full = []

class_file = open(ipf + 'classes.txt','r')
class_no = 0
for line in class_file:
	class_no += 1

for ip in train_data.values:

  one_hot_dest = [0] * class_no
  one_hot_dest[int(ip[1])] = 1
  y_train_full.append(one_hot_dest)
  txt = ip[2].replace('\n','.')
  sents = re.split('[.?!]',txt)
  sents1 = []
  # sents1 = [s for s in sents if len(s) > 1]
  for s in sents:
    for ss in s.split('<br />'):
      # ss = s
      if len(ss) > 2:
        # print('ss = ',ss)
        sents1.append(ss)
  # sents = [s.split('<br />') for s in sents]
  sents2 = ".".join(sents1)
  train_data1.append(sents2)
  sent_count = 0
  # print('sents = ',sents)
  for sent in sents2.split("."):
    if len(sent) < 4:
       continue
    if len(sent.split()) > max_sent_len:
      max_sent_len = len(sent.split())
    if len(sent.split()) >= 200:
      print('long sent = ',sent)
    sent_lens.append(len(sent.split()))
    sent_count+=1
    if sent_count > max_sent_count:
      max_sent_count = sent_count
  sent_counts.append(sent_count)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train_data['review1'] = train_data1

print('train_data = ',train_data)

train_review = train_data.review1



x_review_train, y_train = list(train_review.values), np.array(y_train_full)

train_clean = []
train_not_clean = []
valid_not_clean = []
for review in x_review_train:
    train_clean.append(clean_text(review,remove_stopwords=False))
    train_not_clean.append(review)

print("train_not_clean = ",train_not_clean[:5])
print("train_clean = ",train_clean[:5])

with open(opf + 'train_not_clean', 'wb') as fp:
	pickle.dump(train_not_clean, fp)



with open(opf + 'train_clean', 'wb') as fp:
	pickle.dump(train_clean, fp)

with open(opf + 'y_train', 'wb') as fp:
	pickle.dump(y_train, fp)



print('done')

train_data =          id  ...                                            review1
0        1  ...  Stuning even for the non-gamer. This sound tra...
1        2  ...  The best soundtrack ever to anything. I'm read...
2        3  ...  Amazing. This soundtrack is my favorite music ...
3        4  ...  Excellent Soundtrack. I truly like this soundt...
4        5  ...  Remember, Pull Your Jaw Off The Floor After He...
...    ...  ...                                                ...
2995  2996  ...  Not worth the 3 month wait. 3D is all about po...
2996  2997  ...  garbage. Transformers the Movie (1985) was awe...
2997  2998  ...  If you have already watched trasformer 1, don'...
2998  2999  ...  Transformers. This item was suppose to contain...
2999  3000  ...  Same as all the others, dull. Same as all the ...

[3000 rows x 4 columns]
train_not_clean =  ['Stuning even for the non-gamer. This sound track was beautiful. It paints the senery in your mind so well I would recomend it even to pe

In [ ]:
print(train_clean[0])
print(len(train_clean))

stuning even for the non gamer. this sound track was beautiful. it paints the senery in your mind so well i would recomend it even to people who hate vid. game music. i have played the game chrono cross but out of all of the games i have ever played it has the best music. it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. it would impress anyone who cares to listen. 
3000


# Create sentence embedding vectors

In [ ]:
import pickle
from sentence_transformers import SentenceTransformer, LoggingHandler


ipf = './'

# Load the data
with open (ipf + '/train_clean', 'rb') as fp:
    train_clean = pickle.load(fp)


print(train_clean[:5])
print(train_clean[0])

train_sents, valid_sents, my_test_sents = [], [], []

def gen_sents(para):
	res = []
	for p in para:
		sents = p.split(".")
		res.append(sents)
	return res

train_sents = gen_sents(train_clean)


def process_sents(sents):
	count1, count2 = 0, 0
	sent_list = []
	p_to_sent = {}
	sent_to_p = {}
	for p in sents:
		p_to_sent[count2] = []
		for sent in p:
			sent_list.append(sent)
			p_to_sent[count2].append(count1)
			sent_to_p[count1] = count2
			count1+=1
		count2+=1
	return sent_list, p_to_sent, sent_to_p



['stuning even for the non gamer. this sound track was beautiful. it paints the senery in your mind so well i would recomend it even to people who hate vid. game music. i have played the game chrono cross but out of all of the games i have ever played it has the best music. it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. it would impress anyone who cares to listen. ', 'the best soundtrack ever to anything. i m reading a lot of reviews saying that this is the best game soundtrack and i figured that i d write a review to disagree a bit. this in my opinino is yasunori mitsuda s ultimate masterpiece. the music is timeless and i m been listening to it for years now and its beauty simply refuses to fade.the price tag on this is pretty staggering i must say but if you are going to buy any cd for this much money this is the only one that i feel would be worth every penny', 'amazing. this soundtrack is my favorite music of all time hands 

In [ ]:
train_sent_list, train_p_to_sent, train_sent_to_p = process_sents(train_sents)


print("train_sent_list len = ",len(train_sent_list))
#
def get_test_batches(x, batch_size):
    '''Create the batches for the testing data'''
    n_batches = len(x)//batch_size
    x = x[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size]

train_sent_list len =  18832


In [ ]:
print('----generating----')
print('my_test_sents = ',my_test_sents)

model = SentenceTransformer('bert-base-nli-mean-tokens')


def gen_sent_embed(sent_list, p_to_sent,fname):

		print('generating...)')
		sentence_embeddings = model.encode(sent_list)
		print('sentences_embeddings = ',len(sentence_embeddings))

		with open(fname, 'wb') as fp:
			pickle.dump(sentence_embeddings, fp)

		res = sentence_embeddings

		results = []
		for i in range(len(p_to_sent)):
			res1 = []
			for k in p_to_sent[i]:
				res1.append(res[k])
			results.append(res1)
		
		return results


train_vects = gen_sent_embed(train_sent_list, train_p_to_sent,'train_sentences_embeddings')

print('finished, dumping training vectors...')

with open(ipf + '/train_vects', 'wb') as fp:
		pickle.dump(train_vects, fp)



print('done')

----generating----
my_test_sents =  []



generating...)
sentences_embeddings =  18832
finished, dumping training vectors...
done


# Prepare training data and test data

In [ ]:
#load data
print('----preparing data------')

with open ("train_vects", 'rb') as fp:
  train_vects = pickle.load(fp)

with open ("y_train", 'rb') as fp:
  y_train = pickle.load(fp)

with open ('train_clean', 'rb') as fp:
    train_clean = pickle.load(fp)

with open ('train_not_clean', 'rb') as fp:
    train_not_clean = pickle.load(fp)

y_train_full = np.array(y_train)
y_train = np.array(y_train)
# y_train = np.expand_dims(y_train, axis=1)

----preparing data------


In [ ]:
print("y_train = ",y_train[:20],len(y_train))

y_train =  [[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]] 3000


In [ ]:
data_size = 3000
# valid_size = 1000
def get_batches(x, y,batch_size):
    '''Create the batches for the training and validation data'''
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]
print(len(train_vects), len(train_vects[1]))
print(len(y_train))
print(y_train[0])

num_folds = 5
kFold = 0

X_train_folds = np.array_split(train_vects[:data_size], num_folds)
y_train_folds = np.array_split(y_train[:data_size], num_folds)

x_train, x_valid = [], list(X_train_folds[kFold])
y_train, y_valid = [], list(y_train_folds[kFold])
for i in range(0,num_folds):
    if i != kFold:
        x_train.extend(X_train_folds[i])
        y_train.extend(y_train_folds[i])


print('x_train = ',len(x_train),len(x_train[1]),len(x_train[1][0]))
def check_dist(a):
  return np.array([np.percentile(a,i) for i in [0,25,50,75,100]])
def np_distances(cents,points):
  p1 = np.tile(points, (cents.shape[0], 1)).reshape(cents.shape[0],points.shape[0],points.shape[1])
  c1 = np.expand_dims(cents,axis=1)
  d1 = p1 - c1
  d2 = d1*d1
  d3 = np.sum(d2,axis=2)
  return d3
def match_sents(k_cents,all_sents):
  # res = []
  dProsSents = np_distances(k_cents,np.array(all_sents))
  dProsSentsArgSort = np.argsort(dProsSents,axis=1)
  return all_sents[dProsSentsArgSort[:,0]]

print(check_dist([-5,2,10,0,1,5,4,6,7,-4,-3,8]))
print(check_dist([-1,0.5,1,0.7,-0.8,-0.5,-0.2,0.8, 0.3]))
# print('y_train: ',y_train.shape)

3000 5
3000
[0 1]
x_train =  2400 2 768
[-5.   -0.75  3.    6.25 10.  ]
[-1.  -0.5  0.3  0.7  1. ]


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
#----clustering-----
all_sents = [sent for para in x_train for sent in para]
k_protos = 20
print(len(all_sents),len(all_sents[0]))
kmedoids = KMedoids(n_clusters=k_protos,random_state=0).fit(all_sents[0:10000])
# kmedoids = KMedoids(n_clusters=k_protos,metric='cosine',random_state=0).fit(all_sents[0:10000])
k_cents = kmedoids.cluster_centers_
print(k_cents.shape)

14953 768
(20, 768)


In [ ]:
for c in k_cents:
  print(check_dist(c))

[-2.21361995 -0.36394681 -0.00421329  0.35230746  2.24960947]
[-2.24929452 -0.36212797  0.01551502  0.32109563  1.78031957]
[-1.85124171 -0.34618898  0.00813627  0.33220209  2.21757531]
[-3.07907557e+00 -3.41143601e-01 -2.45144032e-03  3.15083213e-01
  1.84032679e+00]
[-1.86753607 -0.38603641 -0.00364673  0.36949205  1.88630795]
[-1.89380944e+00 -4.50098209e-01 -2.30908592e-03  3.88980240e-01
  2.44710231e+00]
[-3.0378201  -0.36395448 -0.02061718  0.33252378  2.4114027 ]
[-2.53508806 -0.32304862 -0.00699554  0.35568858  2.17692804]
[-2.37569475 -0.37786739 -0.0045268   0.36098149  2.08828211]
[-2.32970643 -0.36985786 -0.01914498  0.31179411  2.02894187]
[-2.51671743 -0.34546052  0.01047658  0.35073602  1.87700117]
[-1.77577376 -0.33705288 -0.03569034  0.30009956  1.81651318]
[-2.23104429e+00 -3.33338112e-01  2.06757849e-03  3.21545132e-01
  1.72285533e+00]
[-2.0619452  -0.38023028  0.01769385  0.35381474  1.52218378]
[-2.70944905e+00 -3.49740244e-01 -3.52017087e-04  3.37844856e-01
  2.

# Create model

In [ ]:
def lstm_cell(lstm_size, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    return drop

In [ ]:
# The default parameters of the model
# n_words = len(word_index)
n_words = 1
embed_size = 300
batch_size = 1
lstm_size = 128
num_layers = 2
dropout = 0.5
learning_rate = 0.01
epochs = 100
multiple_fc = False
fc_units = 256
vect_size = 768

tf.reset_default_graph()

# Declare placeholders we'll feed into the graph
with tf.name_scope('inputs'):
    inputs = tf.placeholder(tf.float32, [batch_size, None, vect_size], name='inputs')
    print('[db] inputs = ', inputs)

with tf.name_scope('labels'):
    labels = tf.placeholder(tf.int32, [batch_size, None], name='labels')

    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

# Build the RNN layers
with tf.name_scope("RNN_layers"):
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(lstm_size, keep_prob) for _ in range(num_layers)])

# Set the initial state
with tf.name_scope("RNN_init_state"):
    initial_state = cell.zero_state(batch_size, tf.float32)

# prototypes
with tf.name_scope("prototypes"):
    protos_values = tf.placeholder(tf.float32, [k_protos, vect_size], name='protos_values')

    prototypes = tf.get_variable('prototypes', shape=[k_protos, vect_size],
                                 initializer=tf.constant_initializer(k_cents[:k_protos]), trainable=True)
    print('[db] prototypes = ', prototypes)

    proto_assign_op = prototypes.assign(protos_values)
    print('[db] proto_assign_op = ', proto_assign_op)



def compute_distance(outputs, p, n_protos):
    tmp1 = tf.expand_dims(outputs, 2)
    tmp1 = tf.broadcast_to(tmp1, [tmp1.shape[0].value, tf.shape(tmp1)[1], n_protos, vect_size])

    tmp2 = tf.broadcast_to(p, [tmp1.shape[0].value, tf.shape(tmp1)[1], n_protos, vect_size])

    tmp3 = tmp1 - tmp2
    tmp4 = tmp3 * tmp3
    distances = tf.reduce_sum(tmp4, axis=3)
    return distances, tf.reduce_min(distances, axis=1)

def e_func(x,a = 0.01, e=2.7182818284590452353602874713527):
    return tf.math.pow(e,-(a*x))

full_distances, full_min_dist = compute_distance(inputs, prototypes, k_protos)

beta = 1e6
min_dist_ind = tf.nn.softmax(-full_distances  * beta)

# dist_hot_vect = min_dist_ind * e_func(tf.math.sqrt(full_distances + 1e-8))
dist_hot_vect = min_dist_ind * e_func(tf.math.sqrt(full_distances + 1e-8))

with tf.name_scope("RNN_forward"):

    outputs, final_state = tf.nn.dynamic_rnn(cell, dist_hot_vect, initial_state=initial_state)


# Create the fully connected layers
with tf.name_scope("fully_connected"):
    # Initialize the weights and biases
    weights = tf.truncated_normal_initializer(stddev=0.1)
    biases = tf.zeros_initializer()

    dense = tf.contrib.layers.fully_connected(outputs[:, -1], num_outputs=fc_units, activation_fn=tf.sigmoid,
                                              weights_initializer=weights, biases_initializer=biases)
    dense = tf.contrib.layers.dropout(dense, keep_prob)

    # Depending on the iteration, use a second fully connected layer
    if multiple_fc == True:
        dense = tf.contrib.layers.fully_connected(dense, num_outputs=fc_units, activation_fn=tf.sigmoid,
                                                  weights_initializer=weights, biases_initializer=biases)
        dense = tf.contrib.layers.dropout(dense, keep_prob)

# Make the predictions
with tf.name_scope('predictions'):
    predictions = tf.contrib.layers.fully_connected(dense,
                                                    num_outputs=2,
                                                    activation_fn=tf.sigmoid,
                                                    weights_initializer=weights,
                                                    biases_initializer=biases)
    predictions = predictions + 1e-8
    tf.summary.histogram('predictions', predictions)

# Calculate the cost
with tf.name_scope('cost'):
    
    def tight_pos_sigmoid_offset(x, offset, e=2.7182818284590452353602874713527):
	      return 1 / (1 + tf.math.pow(e, (1 * (offset*x-0.5))))


    squared_cost = tf.losses.mean_squared_error(labels, predictions) + 1e-8

    def pw_distance(A):
        # A = tf.constant([[1, 1], [4, 4], [5, 5],[6,6]])
        r = tf.reduce_sum(A * A, 1)
        print('[db] r = ', r)
        # turn r into column vector
        r = tf.reshape(r, [-1, 1])
        D = r - 2 * tf.matmul(A, tf.transpose(A)) + tf.transpose(r)
        return D


    costR_d = tf.reduce_sum(tf.reduce_min(full_distances,axis=2))
    
    d = pw_distance(prototypes ) 
    
    diag_ones = tf.convert_to_tensor(np.eye(k_protos, dtype=float))
    diag_ones = tf.dtypes.cast(diag_ones, tf.float32)
    d1 = d + diag_ones * tf.reduce_max(d) 
    d2 = tf.reduce_min(d1, axis=1) 
    min_d2_dist = tf.reduce_min(d2) 
    costC = tight_pos_sigmoid_offset(min_d2_dist, 1) + 1e-8

    cost = squared_cost + 0.0001*costR_d + 0.1*costC


    
    test_var = [full_distances,min_dist_ind,e_func(full_distances),dist_hot_vect]
    # test_var = [costC]


# Train the model
with tf.name_scope('train'):
    lr1, lr2 = 0.0001, 0.0005
    optimizer = tf.train.AdamOptimizer(lr1).minimize(cost)

    # optimizer1 = tf.train.AdamOptimizer(lr1).minimize(cost1)
    # optimizer2 = tf.train.AdamOptimizer(lr2).minimize(cost2)

# Determine the accuracy
with tf.name_scope("accuracy"):
        correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels)
        all_labels_true = tf.reduce_min(tf.cast(correct_pred, tf.float32),1)
        # accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        accuracy = tf.reduce_mean(all_labels_true)

[db] inputs =  Tensor("inputs/inputs:0", shape=(1, ?, 768), dtype=float32)
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
[db] prototypes =  <tf.Variable 'prototypes_1:0' shape=(20, 768) dtype=float32_ref>
[db] proto_assign_op =  Tensor("prototypes/Assign:0", shape=(20, 768), dtype=float32_ref)
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:

# Train the model

In [ ]:
saver = tf.train.Saver()
log_string = 'ru={},fcl={},fcu={}'.format(lstm_size,
                                                      multiple_fc,
                                                      fc_units)   
recorded_train_acc, recorded_valid_acc = [], []     
with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        valid_loss_summary = []
        
        # Keep track of which batch iteration is being trained
        iteration = 0

        avg_train_acc = 0
        
        with open('_prototypes', 'wb') as fp:
          pickle.dump(k_cents, fp)
        with open ('_prototypes', 'rb') as fp:
          _prototypes = pickle.load(fp)
              
        for e in range(epochs):
            state = sess.run(initial_state)
            
            # Record progress with each epoch
            train_loss, train_acc, val_acc, val_loss = [], [], [], []
            # optimizer = optimizer1
            print('[db] Using opt1')
            # if avg_train_acc > 0.65:
            #   print('[db] Using opt2')
            #   optimizer = model.optimizer2

            with tqdm(total=len(x_train)) as pbar:
                for i, (x, y) in enumerate(get_batches(x_train, y_train, batch_size), 1):
                    # print("x = ",len(x),len(x[0]))
                    # print("y = ",y)
                    feed = {inputs: x,
                            protos_values: _prototypes,      
                            labels: y,
                            keep_prob: dropout,
                            initial_state: state,
                            }
                    loss, \
                    acc, \
                    state,\
                    _, \
                    _prototypes, \
                    _predictions, \
                    _test_var = sess.run([
                                                             cost, 
                                                             accuracy, 
                                                             final_state, 
                                                             optimizer,
                                                             prototypes,
                                                             predictions,
                                                             test_var
                                                            ], 
                                                           feed_dict=feed)
    
                    if i % 1000 == 0:
                      # print("test_var: ",i)
                      # for v in _test_var:
                      #   print('v = ',v,v.shape)
                      with open('./test_var', 'wb') as fp:
                        pickle.dump(_test_var, fp)
                      # print("_neg_prototypes : ",_neg_prototypes)
                    # Record the loss and accuracy of each training batch
                    train_loss.append(loss)
                    train_acc.append(acc)                  
                    # Record the progress of training
                    
                    iteration += 1
                    pbar.update(batch_size)

            # --- projection
            if e % 10 == 0:
              new_protos = match_sents(_prototypes,np.array(all_sents[:10000]))

              _prototypes, _prototypes_values, _  = sess.run([prototypes, protos_values, proto_assign_op], feed_dict={protos_values: new_protos})

              print(_prototypes)
              print('Saving projected protos...')
              with open('_prototypes', 'wb') as fp:
                    pickle.dump(_prototypes, fp)


              checkpoint = "./sentiment_{}.ckpt".format(log_string)
              saver.save(sess, checkpoint)
            # -----

            # Average the training loss and accuracy of each epoch
            avg_train_loss = np.mean(train_loss)
            avg_train_acc = np.mean(train_acc)
            
            recorded_train_acc.append(avg_train_acc)

            val_state = sess.run(initial_state)
            with tqdm(total=len(x_valid)) as pbar:
                for x, y in get_batches(x_valid, y_valid, batch_size):
                    # print("y = ",y)
                    feed = {inputs: x,
                            protos_values: _prototypes,
                            # model.labels: np.squeeze(y,axis=2),
                            labels: y,
                            keep_prob: 1,
                            initial_state: val_state,
                            # model.pretrained_embedding: embedding_matrix
                            }
                    batch_loss, \
                    batch_acc, \
                    val_state, \
                    _prototypes, \
                    _predictions  = sess.run([ 
                                                                          cost, 
                                                                          accuracy, 
                                                                          final_state,
                                                                          prototypes,
                                                                          predictions
                                                                          ], 
                                                                         feed_dict=feed)
                    
                    # Record the validation loss and accuracy of each epoch
                    val_loss.append(batch_loss)
                    val_acc.append(batch_acc)

                    pbar.update(batch_size)
            
            # Average the validation loss and accuracy of each epoch
            avg_valid_loss = np.mean(val_loss)    
            avg_valid_acc = np.mean(val_acc)

            recorded_valid_acc.append(avg_valid_acc)


            valid_loss_summary.append(avg_valid_loss)
            
            # Record the validation data's progress

            print("Epoch: {}/{}".format(e, epochs))
            print("Train Loss: {:.3f}".format(avg_train_loss))
            print("Train Acc: {:.3f}".format(avg_train_acc))
            print("Valid Loss: {:.3f}".format(avg_valid_loss))
            print("Valid Acc: {:.3f}".format(avg_valid_acc))

            with open('_recorded_train_acc', 'wb') as fp:
                    pickle.dump(recorded_train_acc, fp)
            with open('_recorded_valid_acc', 'wb') as fp:
                    pickle.dump(recorded_valid_acc, fp)

            # Stop training if the validation loss does not decrease after 3 epochs
            if avg_valid_loss > min(valid_loss_summary):
                print("No Improvement.")
                stop_early += 1
                if stop_early == 100:
                    break   
            
            # Reset stop_early if the validation loss finds a new low
            # Save a checkpoint of the model
            else:
                print("New Record!")
                stop_early = 0
                checkpoint = "./sentiment_{}.ckpt".format(log_string)
                saver.save(sess, checkpoint)

  0%|          | 0/2400 [00:00<?, ?it/s]

[db] Using opt1


  0%|          | 0/600 [00:00<?, ?it/s]

[[ 0.10788302 -0.07812521  1.3328316  ... -0.32942432 -0.4494228
   0.10711775]
 [ 0.11242216 -0.10783485  0.611016   ...  0.09269011 -0.35730195
   0.6045902 ]
 [ 0.2713949   1.0684483   1.1182355  ...  0.3879594  -0.37539917
   0.24002054]
 ...
 [-0.32964328  0.6073361   1.1103657  ...  0.35603926  0.16909082
  -0.23876734]
 [ 0.27703696 -0.02003925  1.449973   ...  0.39265326 -0.94815385
   0.15352663]
 [-0.0067452   0.5994327   0.3586586  ...  0.3032902  -0.10959001
   0.09642712]]
Saving projected protos...


  0%|          | 0/2400 [00:00<?, ?it/s]

...

Epoch: 6/100
Train Loss: 0.119
Train Acc: 0.810
Valid Loss: 0.119
Valid Acc: 0.835
No Improvement.
[db] Using opt1


  0%|          | 0/2400 [00:00<?, ?it/s]

Epoch: 7/100
Train Loss: 0.115
Train Acc: 0.823
Valid Loss: 0.116
Valid Acc: 0.835
New Record!
[db] Using opt1


 37%|███▋      | 885/2400 [00:14<00:25, 59.15it/s]


KeyboardInterrupt: ignored

# Show the prototypes

In [ ]:
with open ('./_prototypes', 'rb') as fp:
	_prototypes = pickle.load(fp)

In [ ]:
print('protos: ',_prototypes.shape)
for p in _prototypes:
  print(check_dist(p))
print('y_train_full: ',y_train_full.shape)

protos:  (20, 768)
[-2.19912577e+00 -3.57001521e-01 -4.07308107e-05  3.46618354e-01
  2.23181701e+00]
[-2.25397158 -0.36350631  0.01726294  0.3257924   1.78636467]
[-1.85179901 -0.34470259  0.00880165  0.33422242  2.22352934]
[-3.06750512e+00 -3.39656420e-01  1.70232449e-03  3.19128118e-01
  1.83088076e+00]
[-1.87267697 -0.38817275 -0.00448445  0.3662883   1.89859831]
[-1.88488901 -0.43287193 -0.00490439  0.36751947  2.36349893]
[-3.03457737 -0.36203022 -0.02228838  0.33183572  2.41213989]
[-2.49775696 -0.31603245 -0.01041282  0.34754016  2.15590715]
[-2.37612534 -0.3777848  -0.00519938  0.36035922  2.08403993]
[-2.33463454 -0.36823773 -0.01960657  0.31087359  2.02590632]
[-2.49007773 -0.34609889  0.00912199  0.34327395  1.85478377]
[-1.7729826  -0.33799652 -0.03572526  0.29794648  1.81905711]
[-2.21585298 -0.3307818  -0.00253093  0.31366517  1.69526482]
[-2.05411577 -0.37290997  0.02154736  0.35185701  1.51402354]
[-2.68696332 -0.35192186 -0.00556442  0.3407717   2.0243783 ]
[-2.72079

In [ ]:
print(len(x_train),len(y_train))

2400 2400


In [ ]:
p_count = 0
d_pos = {}
for p_count,p in enumerate(_prototypes):
  print('p_count = ',p_count)
  s_count = 0
  d_pos[p_count] = {}
  for i,para in enumerate(train_vects[:data_size]):
    for j,s in enumerate(para):
      if len(train_clean[i].split(".")[j]) < 5:
        continue
      d_pos[p_count][(i,j)] = np.linalg.norm(train_vects[i][j]-p)
      # print('i,j = ',i,j)
      # d_pos[p_count][(i,j)] = spatial.distance.cosine(train_vects[i][j], p)
      s_count+=1
  print('count = ',s_count)

p_count =  0
count =  18650
p_count =  1
count =  18650
p_count =  2
count =  18650
p_count =  3
count =  18650
p_count =  4
count =  18650
p_count =  5
count =  18650
p_count =  6
count =  18650
p_count =  7
count =  18650
p_count =  8
count =  18650
p_count =  9
count =  18650
p_count =  10
count =  18650
p_count =  11
count =  18650
p_count =  12
count =  18650
p_count =  13
count =  18650
p_count =  14
count =  18650
p_count =  15
count =  18650
p_count =  16
count =  18650
p_count =  17
count =  18650
p_count =  18
count =  18650
p_count =  19
count =  18650


In [ ]:
import operator
protos_sents = {}
k_closest_sents = 10
recorded_protos_score = {}
for l in range(k_protos):
  print("prototype index = ",l)
  recorded_protos_score[l] = {}
  sorted_d = sorted(d_pos[l].items(), key=operator.itemgetter(1))
  proto_scores = []
  closest_sents = []
  for k in range(k_closest_sents):
    (i,j) = sorted_d[k][0]
    print(sorted_d[k], y_train_full[i], train_not_clean[i].split(".")[j])


prototype index =  0
((2135, 2), 0.33493796) [1 0]  Even when it did work it was a hit or miss thing
((272, 7), 10.132784) [1 0] 95 above and beyond the already rather pricy product
((701, 3), 10.58604) [1 0]  I did, although it was unrewarding to do so
((1367, 5), 10.586723) [1 0]  Once it was on it really didn't do what it is supposed to do - it's more awkward than anything
((2746, 0), 10.61009) [0 1] This is made well and strudy, but will wear out
((562, 8), 10.662851) [0 1] but it DOESNT move around) Totally worth it
((109, 3), 10.670017) [0 1]  You sometimes miss the most obvious
((557, 2), 10.672116) [0 1]  the only issue is the first time you put it on it is pretty difficult to put on
((934, 2), 10.709221) [1 0]  It is crude beyond "good taste
((2592, 5), 10.757979) [1 0]  Unusable after one use
prototype index =  1
((1581, 4), 0.12333472) [1 0] While certain parts of this story are interesting, in it's entirety, i would have to give it two thumbs down (
((2617, 5), 9.726264) [1